## surface temperature (tas)

In [ ]:
# model #3 and #9 have differnt emsemble in historical (r6i1p1)
# model #3 and #9 do not have different emsemble than other models in rcp85


In [ ]:
subset = intake.cat.nci['esgf'].cmip5.search(
                                            model_id = model, 
                                            experiment = experiment,
                                            time_frequency = 'mon', 
                                            realm = 'atmos', 
                                            ensemble = ensemble, 
                                            variable= 'tas'
)
subset

In [ ]:
ds_dict = subset.to_dataset_dict()

In [ ]:
ds_tas = ds_dict[list(ds_dict.keys())[-1]].sel(time=period, lon=slice(0,360),lat=slice(-30,30))
ds_tas

In [ ]:
ds_tas.tas.encoding

In [ ]:
ds_tas = ds_tas.where(ds_tas.tas < 1e+20) 

In [ ]:
tas = regridder(ds_tas).tas-273.15
tas.attrs['units']= 'deg (C)'
tas

In [ ]:
tas_day = tas.isel(time=0)
lat = tas_day.lat
lon = tas_day.lon

In [ ]:
# plot snapshot
projection = ccrs.PlateCarree(central_longitude=180)

f, ax = plt.subplots(subplot_kw=dict(projection=projection), figsize=(15, 5))

tas_day.plot(transform=ccrs.PlateCarree(), cbar_kwargs={'orientation': 'horizontal','pad':0.125, 'aspect':50,'fraction':0.055}, cmap='coolwarm')
ax.add_feature(cfeat.COASTLINE)
ax.set_extent([lon[0], lon[-1], lat[0], lat[-1]], crs=ccrs.PlateCarree())
ax.set_title('Snapshot, model:' + model + ' exp:' + experiment)
ax.set_xticks([-180, -90, 0, 90, 180])
ax.set_xticklabels([0, 90, 180, 270, 360])
ax.set_yticks([-20, 0, 20])
plt.tight_layout()


In [ ]:
tas_tMean= tas.mean(dim='time', keep_attrs=True)

# areaweighting
# weights = np.cos(np.deg2rad(lat))
# weights.name = "weights"
# pr_tMean.weighted(weights).mean(dim=('lat','lon'))

In [ ]:
# plot timeMean
projection = ccrs.PlateCarree(central_longitude=180)

f, ax = plt.subplots(subplot_kw=dict(projection=projection), figsize=(15, 5))

tas_tMean.plot(transform=ccrs.PlateCarree(), cbar_kwargs={'orientation': 'horizontal','pad':0.125, 'aspect':50,'fraction':0.055}, cmap='coolwarm')
ax.add_feature(cfeat.COASTLINE)
ax.set_extent([lon[0], lon[-1], lat[0], lat[-1]], crs=ccrs.PlateCarree())
ax.set_title('Snapshot, model:' + model + ' exp:' + experiment)
ax.set_xticks([-180, -90, 0, 90, 180])
ax.set_xticklabels([0, 90, 180, 270, 360])
ax.set_yticks([-20, 0, 20])
plt.tight_layout()

## saving

In [ ]:
# spatial mean temperature
save = False
if save:
    folder = '/g/data/k10/cb4968/cmip5/' + model
    os.makedirs(folder, exist_ok=True)
    
    fileName = model + '_tas_' + experiment + '.nc'
    path = folder + '/' + fileName
    if os.path.exists(path):
        os.remove(path)    

    xr.Dataset({'tas_day': tas_day, 'tas_tMean': tas_tMean}).to_netcdf(path)